In [1]:
from Crypto.Cipher import AES
import base64
import requests
import json
import codecs
import time

In [2]:
 # 头部信息
headers = {
    'Host':"music.163.com",
    'Accept-Language':"zh-CN,zh;q=0.9",
    'Accept-Encoding':"gzip, deflate, br",
    'Content-Type':"application/x-www-form-urlencoded",
    'Cookie':"_ntes_nuid=dd7f8685455f10fb390d793936928976; mail_psc_fingerprint=6b0bfaa221f15c33b0dd95b106a9f8c9; __gads=ID=75f9f8e082092a55:T=1515223556:S=ALNI_Mb5pjVycWIp9w9x0JDLkmABUGFXog; vjuids=4fdbc64a3.160ca5cc07d.0.e37686b4d5484; _iuqxldmzr_=32; usertrack=ezq0pVrcA1dRXTzdhY05Ag==; _ga=GA1.2.1506570853.1524368226; WM_TID=3tMA1GCrQsEmivmY0iBDQ5ZaWuJGgibp; _ntes_nnid=dd7f8685455f10fb390d793936928976,1538535350025; __oc_uuid=6904d0f0-f31b-11e8-9c76-3ba59817ce49; UM_distinctid=1687fff1db02fe-02db4fe1de6721-323b5b03-100200-1687fff1db13e2; vjlast=1536989069.1548517593.11; mp_MA-9ADA-91BF1A6C9E06_hubble=%7B%22sessionReferrer%22%3A%20%22https%3A%2F%2Fcampus.163.com%2Fapp%2Fhy%2Flh%22%2C%22updatedTime%22%3A%201551605199399%2C%22sessionStartTime%22%3A%201551605199301%2C%22sendNumClass%22%3A%20%7B%22allNum%22%3A%202%2C%22errSendNum%22%3A%200%7D%2C%22deviceUdid%22%3A%20%22daad0ce6-89a8-4980-9e2d-59c9c9868d59%22%2C%22persistedTime%22%3A%201551597516473%2C%22LASTEVENT%22%3A%20%7B%22eventId%22%3A%20%22da_screen%22%2C%22time%22%3A%201551605199400%7D%2C%22sessionUuid%22%3A%20%225b639d67-bd1b-4d1b-96d1-8aa694552878%22%7D; Province=025; City=025; NNSSPID=bf5058e7ef3a4ea58b4e44e456fb15a3; P_INFO=travailamanda@163.com|1553349248|0|mail163|00&99|jis&1553325344&carddav#jis&320100#10#0#0|&0|mail163|travailamanda@163.com; nts_mail_user=travailamanda@163.com:-1:1; _antanalysis_s_id=1553397423109; ne_analysis_trace_id=1553405890144; s_n_f_l_n3=f092a574340d015f1553412813788; vinfo_n_f_l_n3=f092a574340d015f.1.10.1536992799650.1553409996534.1553412815148; WM_NI=N6n6%2FMK0GVCz64PsO%2F2B4PW3NgG85qNZXlNUKuKjno1da%2F4uPof1u3Q%2FEfGYst0fixfFqer4KAu8a7NI3SjtT%2F67wGIJ2g1jPQ3pMZrw%2FLBL6Txv2wJvKAkF6IVmJfufNzc%3D; WM_NIKE=9ca17ae2e6ffcda170e2e6eeb8d234fc98a390e870fc868fa2c44b969f9eaebb6dfceef9aebc598b8ae1acf82af0fea7c3b92a8eeaa2d7ae4789f089b7c272f6eca094ce25f29efcb1d6398eb989d9b87b8e86988dcd5f89ab858bed65f3bcafd0b77f94b1aad2f83993f18b87c653b1ac0087ec74a69abf86f140899fbbd9e43d8e97fdb4e98098f1fad8f64d8b96fe88b462948ffd91d052b2a9e1adc148f68cb897b17d8aaa85a2ea4ef6979a8ec96eb7ba829be237e2a3; JSESSIONID-WYYY=RWBVdrvsQwUslOaDrBUC0EfzWf1t9QYDYQCj6fkP%2BEaibUOxrsIOFxGZ2mBBnvlaW1wV6cNsUN%2Ft8aPv%2FfGpqQ97cyX6HGlNm1p4%2FZN%2Fv%2BCZJsuFXH99MHJQMJtg3rfY1A7xUTrKB%5CYiU%5CwuYh65BdxPy%5CORBBF44%2F4Hn4uWRv%2FEv%5Caa%3A1553435517900",
    'Connection':"keep-alive",
    'Referer':'https://music.163.com/song?id=419594404'
}

In [3]:
# offset的取值为:(评论页数-1)*20,total第一页为true，其余页为false
# first_param = '{rid:"", offset:"0", total:"true", limit:"20", csrf_token:""}' # 第一个参数
second_param = "010001" # 第二个参数
# 第三个参数
third_param = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
# 第四个参数
forth_param = "0CoJUm6Qyw8W8jud"

In [4]:
# 获取参数
def get_params(page): # page为传入页数
    iv = "0102030405060708"
    first_key = forth_param
    second_key = 16 * 'F'
    if(page == 1): # 如果为第一页
        first_param = '{rid:"", offset:"0", total:"true", limit:"20", csrf_token:""}'
        h_encText = AES_encrypt(first_param, first_key, iv)
    else:
        offset = str((page-1)*20)
        first_param = '{rid:"", offset:"%s", total:"%s", limit:"20", csrf_token:""}' %(offset,'false')
        h_encText = AES_encrypt(first_param, first_key, iv)
    h_encText = AES_encrypt(h_encText, second_key, iv)
    return h_encText

In [5]:
 # 获取 encSecKey
def get_encSecKey():
    encSecKey = "257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c"
    return encSecKey

In [6]:
# 解密过程
def AES_encrypt(text, key, iv):
    pad = 16 - len(text) % 16
    text = text + pad * chr(pad)
    encryptor = AES.new(key, AES.MODE_CBC, iv)
    encrypt_text = encryptor.encrypt(text)
    encrypt_text = base64.b64encode(encrypt_text)
    encrypt_text = str(encrypt_text, encoding="utf-8")
    return encrypt_text

In [7]:
# 获得评论json数据
def get_json(url, params, encSecKey):
    data = {
        "params": params,
        "encSecKey": encSecKey
            }
    response = requests.post(url, headers=headers, data=data) # ,proxies = proxies 无法用
    return response.content

In [8]:
# 抓取热门评论，返回热评列表
def get_hot_comments(url):
    hot_comments_list = []
    hot_comments_list.append(u"用户ID 用户昵称 用户头像地址 评论时间 点赞总数 评论内容\n")
    params = get_params(1) # 第一页
    encSecKey = get_encSecKey()
    json_text = get_json(url,params,encSecKey)
    json_dict = json.loads(json_text)
    hot_comments = json_dict['hotComments'] # 热门评论
    print("共有%d条热门评论!" % len(hot_comments))
    for item in hot_comments:
        comment = item['content'] # 评论内容
        likedCount = item['likedCount'] # 点赞总数
        comment_time = item['time'] # 评论时间(时间戳)
        userID = item['user']['userID'] # 评论者id
        nickname = item['user']['nickname'] # 昵称
        avatarUrl = item['user']['avatarUrl'] # 头像地址
        comment_info = userID + " " + nickname + " " + avatarUrl + " " + comment_time + " " + likedCount + " " + comment + u"\n"
        hot_comments_list.append(comment_info)
    return hot_comments_list

In [9]:
# 抓取某一首歌的全部评论
def get_all_comments(url):
    all_comments_list = [] # 存放所有评论
    all_comments_list.append(u"用户ID 用户昵称 用户头像地址 评论时间 点赞总数 评论内容\n") # 头部信息
    params = get_params(1)
    encSecKey = get_encSecKey()
    json_text = get_json(url,params,encSecKey)
    json_dict = json.loads(json_text)
    comments_num = int(json_dict['total'])
    if(comments_num % 20 == 0):
        page = comments_num / 20
    else:
        page = int(comments_num / 20) + 1
    print("共有%d页评论!" % page)
    for i in range(page):  # 逐页抓取
        params = get_params(i+1)
        encSecKey = get_encSecKey()
        json_text = get_json(url,params,encSecKey)
        json_dict = json.loads(json_text)
        if i == 0:
            print("共有%d条评论!" % comments_num) # 全部评论总数
        def parse_json_data(contents):
            if contents:
                contents=contents.decode('utf-8')
                comments=json.loads(contents)['comments']
        for item in json_dict['comments']:
            comment=item['content'] # 评论内容
            likedCount = item['likedCount'] # 点赞总数
            comment_time = item['time'] # 评论时间(时间戳)
            userID = item['user']['userId'] # 评论者id
            nickname = item['user']['nickname'] # 昵称
            avatarUrl = item['user']['avatarUrl'] # 头像地址
            comment_info = str(userID) + u" " + nickname + u" " + avatarUrl + u" " + str(comment_time) + u" " + str(likedCount) + u" " + comment + u"\n"
        all_comments_list.append(comment_info)   # 在python2中，上一条代码str换成Unicode
        print("第%d页抓取完毕!" % (i+1))
    return all_comments_list

In [10]:
# 将评论写入文本文件
def save_to_file(list,filename):
        with codecs.open(filename,'a',encoding='utf-8') as f:
            f.writelines(list)
        print("写入文件成功!")

if __name__ == "__main__":
    start_time = time.time() # 开始时间
    url = "https://music.163.com/weapi/v1/resource/comments/R_SO_4_419594404?csrf_token="
    filename = u"VeryNice.txt"
    all_comments_list = get_all_comments(url)
    save_to_file(all_comments_list,filename)
    end_time = time.time() #结束时间
    print("程序耗时%f秒." % (end_time - start_time))

共有241页评论!
共有4805条评论!
第1页抓取完毕!
第2页抓取完毕!
第3页抓取完毕!
第4页抓取完毕!
第5页抓取完毕!
第6页抓取完毕!
第7页抓取完毕!
第8页抓取完毕!
第9页抓取完毕!
第10页抓取完毕!
第11页抓取完毕!
第12页抓取完毕!
第13页抓取完毕!
第14页抓取完毕!
第15页抓取完毕!
第16页抓取完毕!
第17页抓取完毕!
第18页抓取完毕!
第19页抓取完毕!
第20页抓取完毕!
第21页抓取完毕!
第22页抓取完毕!
第23页抓取完毕!
第24页抓取完毕!
第25页抓取完毕!
第26页抓取完毕!
第27页抓取完毕!
第28页抓取完毕!
第29页抓取完毕!
第30页抓取完毕!
第31页抓取完毕!
第32页抓取完毕!
第33页抓取完毕!
第34页抓取完毕!
第35页抓取完毕!
第36页抓取完毕!
第37页抓取完毕!
第38页抓取完毕!
第39页抓取完毕!
第40页抓取完毕!
第41页抓取完毕!
第42页抓取完毕!
第43页抓取完毕!
第44页抓取完毕!
第45页抓取完毕!
第46页抓取完毕!
第47页抓取完毕!
第48页抓取完毕!
第49页抓取完毕!
第50页抓取完毕!
第51页抓取完毕!
第52页抓取完毕!
第53页抓取完毕!
第54页抓取完毕!
第55页抓取完毕!
第56页抓取完毕!
第57页抓取完毕!
第58页抓取完毕!
第59页抓取完毕!
第60页抓取完毕!
第61页抓取完毕!
第62页抓取完毕!
第63页抓取完毕!
第64页抓取完毕!
第65页抓取完毕!
第66页抓取完毕!
第67页抓取完毕!
第68页抓取完毕!
第69页抓取完毕!
第70页抓取完毕!
第71页抓取完毕!
第72页抓取完毕!
第73页抓取完毕!
第74页抓取完毕!
第75页抓取完毕!
第76页抓取完毕!
第77页抓取完毕!
第78页抓取完毕!
第79页抓取完毕!
第80页抓取完毕!
第81页抓取完毕!
第82页抓取完毕!
第83页抓取完毕!
第84页抓取完毕!
第85页抓取完毕!
第86页抓取完毕!
第87页抓取完毕!
第88页抓取完毕!
第89页抓取完毕!
第90页抓取完毕!
第91页抓取完毕!
第92页抓取完毕!
第93页抓取完毕!
第94页抓取完毕!
第95页抓取完毕!
第96页抓取完毕!
第97页抓取完毕!
第98页抓取完毕!
第99页抓取完毕